# import packages

In [1]:
import random

import Bio 
from Bio import SeqIO

# Class cotaining usefull methods
## Basic DNA Sequence Analysis

In [2]:
class DNAToolkit:
    """Basic DNA sequence analysis tools"""
    
    def __init__(self, sequence):
        """Initialize with DNA sequence"""
        self.sequence = sequence.upper()  # Standardize input to uppercase
        self.valid_bases = set('ATCG')    # Set for O(1) lookup time
        
    def validate_sequence(self):
        """Check if sequence contains only valid DNA bases"""
        # all() with generator expression is memory efficient for long sequences
        return all(base in self.valid_bases for base in self.sequence)
    
    def sequence_length(self):
        """Return sequence length"""
        return len(self.sequence)
    
    def nucleotide_count(self):
        """Count occurrences of each nucleotide"""
        # Direct count is more readable than Counter for learning
        return {
            'A': self.sequence.count('A'),
            'T': self.sequence.count('T'),
            'G': self.sequence.count('G'),
            'C': self.sequence.count('C')
        }
    
    def gc_content(self):
        """Calculate GC content percentage"""
        counts = self.nucleotide_count()
        gc = counts['G'] + counts['C']    # Sum G and C counts
        total = sum(counts.values())       # Total bases
        return (gc / total) * 100 if total > 0 else 0  # Handle empty sequence
    
    def find_start_codons(self):
        """Find all start codon (ATG) positions"""
        positions = []
        # Step through sequence with 3-base window
        # -2 ensures enough room for complete codon
        for i in range(0, len(self.sequence) - 2):
            if self.sequence[i:i+3] == 'ATG':  # Start codon check
                positions.append(i + 1)         # Convert to 1-based position
        return positions
    
    def find_stop_codons(self):
        """Find all stop codon (TAA, TAG, TGA) positions"""
        stop_codons = ['TAA', 'TAG', 'TGA']    # All possible stop codons
        positions = []
        # Same window sliding as start codons
        for i in range(0, len(self.sequence) - 2):
            if self.sequence[i:i+3] in stop_codons:
                positions.append(i + 1)
        return positions
    
    def reverse_complement(self):
        """Generate reverse complement of sequence"""
        # DNA base pairing rules
        complement_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
        # Build complement then reverse it
        complement = ''.join(complement_dict[base] for base in self.sequence)
        return complement[::-1]  # Slice notation for reverse

# FASTA File Analysis (Using BioPython)

In [3]:
from Bio import SeqIO

class FastaAnalyzer:
    """FASTA file analysis tools"""
    
    def __init__(self, fasta_file):
        """Initialize with FASTA file path"""
        self.fasta_file = fasta_file
        self.records = list(SeqIO.parse(fasta_file, "fasta"))
        
    def count_records(self):
        """Count number of records in FASTA file"""
        return len(self.records)
    
    def get_sequence_lengths(self):
        """Get dictionary of sequence IDs and their lengths"""
        return {record.id: len(record.seq) for record in self.records}
    
    def find_extreme_sequences(self):
        """Find longest and shortest sequences"""
        if not self.records:
            return None
            
        lengths = self.get_sequence_lengths()
        min_len = min(lengths.values())
        max_len = max(lengths.values())
        
        shortest = [id for id, length in lengths.items() if length == min_len]
        longest = [id for id, length in lengths.items() if length == max_len]
        
        return {
            'shortest': {'length': min_len, 'ids': shortest},
            'longest': {'length': max_len, 'ids': longest}
        }

# ORF Finding

In [4]:
class ORFFinder:
    """Open Reading Frame analysis tools"""
    
    def __init__(self, sequence):
        """Initialize with DNA sequence"""
        self.sequence = sequence.upper()
        
    def get_reading_frame(self, frame):
        """Get sequence in specified reading frame (1, 2, or 3)"""
        # frame-1 converts 1-based frame to 0-based index
        return self.sequence[frame-1:]
    
    def find_orfs_in_frame(self, frame):
        """Find all ORFs in specified reading frame"""
        seq = self.get_reading_frame(frame)
        orfs = []
        
        i = 0
        # -5 ensures room for both start and stop codons
        while i < len(seq) - 5:
            if seq[i:i+3] == 'ATG':    # Found potential start
                j = i + 3              # Move to next codon
                while j < len(seq) - 2:
                    codon = seq[j:j+3]
                    if codon in ['TAA', 'TAG', 'TGA']:  # Found stop
                        # Record ORF details with frame-adjusted positions
                        orfs.append({
                            'start': i + frame,
                            'end': j + frame + 2,
                            'sequence': seq[i:j+3],
                            'length': j + 3 - i
                        })
                        break
                    j += 3  # Move to next codon
            i += 3  # Move to next potential start
        return orfs
    
    def find_all_orfs(self):
        """Find ORFs in all three forward frames"""
        all_orfs = []
        # Check each possible reading frame
        for frame in [1, 2, 3]:
            all_orfs.extend(self.find_orfs_in_frame(frame))
        return all_orfs
    
    def get_longest_orf(self):
        """Find the longest ORF across all frames"""
        orfs = self.find_all_orfs()
        if not orfs:
            return None
        # Use key function with max for efficiency
        return max(orfs, key=lambda x: x['length'])

# DNA Repeat Finder

In [5]:
class RepeatFinder:
    """DNA repeat sequence analysis tools"""
    
    def __init__(self, sequence):
        self.sequence = sequence.upper()
        
    def find_repeats(self, length):
        """Find all repeats of specified length"""
        repeats = {}
        
        # Generate all possible substrings of given length
        # +1 ensures last substring is included
        for i in range(len(self.sequence) - length + 1):
            substring = self.sequence[i:i+length]
            count = 0
            
            # Count all occurrences of current substring
            for j in range(len(self.sequence) - length + 1):
                if self.sequence[j:j+length] == substring:
                    count += 1
                    
            # Store only if it's a repeat (count > 1)
            if count > 1:
                repeats[substring] = count
                
        return repeats
    
    def most_frequent_repeat(self, length):
        """Find most frequent repeat of specified length"""
        repeats = self.find_repeats(length)
        if not repeats:
            return None
        
        # Find highest count
        max_count = max(repeats.values())
        # Get all sequences with that count
        most_frequent = [seq for seq, count in repeats.items() 
                        if count == max_count]
        
        return {
            'sequences': most_frequent,
            'count': max_count
        }

# Example Usage

In [7]:
# Example DNA sequence analysis
dna_seq = "ATGGCCTAAGGCTGATAGATGGCCTAA"
toolkit = DNAToolkit(dna_seq)

print(f"Sequence Length: {toolkit.sequence_length()}")
print(f"Nucleotide Counts: {toolkit.nucleotide_count()}")
print(f"GC Content: {toolkit.gc_content():.2f}%")
print(f"Start Codons at: {toolkit.find_start_codons()}")
print(f"Stop Codons at: {toolkit.find_stop_codons()}")
print(f"Reverse Complement: {toolkit.reverse_complement()}")

# Example FASTA analysis
fasta_analyzer = FastaAnalyzer("dna2.fasta")
print(f"Number of Records: {fasta_analyzer.count_records()}")
print(f"Sequence Lengths: {fasta_analyzer.get_sequence_lengths()}")
print(f"Extreme Sequences: {fasta_analyzer.find_extreme_sequences()}")

# Example ORF finding
orf_finder = ORFFinder(dna_seq)
print(f"Longest ORF: {orf_finder.get_longest_orf()}")

# Example repeat finding
repeat_finder = RepeatFinder(dna_seq)
print(f"Repeats of length 3: {repeat_finder.find_repeats(3)}")
print(f"Most frequent repeat: {repeat_finder.most_frequent_repeat(3)}")

Sequence Length: 27
Nucleotide Counts: {'A': 8, 'T': 6, 'G': 8, 'C': 5}
GC Content: 48.15%
Start Codons at: [1, 19]
Stop Codons at: [7, 13, 16, 25]
Reverse Complement: TTAGGCCATCTATCAGCCTTAGGCCAT
Number of Records: 18
Sequence Lengths: {'gi|142022655|gb|EQ086233.1|91': 4635, 'gi|142022655|gb|EQ086233.1|304': 1151, 'gi|142022655|gb|EQ086233.1|255': 4894, 'gi|142022655|gb|EQ086233.1|45': 3511, 'gi|142022655|gb|EQ086233.1|396': 4076, 'gi|142022655|gb|EQ086233.1|250': 2867, 'gi|142022655|gb|EQ086233.1|322': 442, 'gi|142022655|gb|EQ086233.1|88': 890, 'gi|142022655|gb|EQ086233.1|594': 967, 'gi|142022655|gb|EQ086233.1|293': 4338, 'gi|142022655|gb|EQ086233.1|75': 1352, 'gi|142022655|gb|EQ086233.1|454': 4564, 'gi|142022655|gb|EQ086233.1|16': 4804, 'gi|142022655|gb|EQ086233.1|584': 964, 'gi|142022655|gb|EQ086233.1|4': 2095, 'gi|142022655|gb|EQ086233.1|277': 1432, 'gi|142022655|gb|EQ086233.1|346': 115, 'gi|142022655|gb|EQ086233.1|527': 2646}
Extreme Sequences: {'shortest': {'length': 115, 'ids': 

# Useful function

In [6]:
#Create random string
def random_string_generator(length):
    import random
    string = "" 
    desired_length = length
    for _ in range(desired_length):
        string += random.choice('ACTG')
    return string
print(random_string_generator(10))

GCTCGCATCT


In [7]:
# Find longoest Common prefix

def lcprefix(s1, s2):
    i = 0
    while i < len(s1) and i < len(s2) and s1[i] == s2[i]:
        i += 1
    return s1[:i]

print(lcprefix("acctvfd", "acctkfj"))

acct


## Read genome, fasta file


In [15]:
def read_genome(file):
    genome = ""
    with open(file, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome = genome + line.rstrip()
    return genome

genome = read_genome("dna2.fasta")
genome[:100]

'CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGGGAGGATCTCGGCGGCGCCAACTATGCGGTCTTTCGGCTCGAAAGCC'

## count number of bases

In [18]:
def count_bases(genome_seq, specific_base = None):
    """
    This function calculate the # of occurence of each base.
    :param genome_seq: 
    :param specific_base: 
    :return: count_dict:
    """
    # we can also use collections module to do the samething.
    count_dict = {}
    for base in genome_seq:
        if base not in count_dict:
            count_dict[base] = 1
        else:
            count_dict[base] += 1
    if specific_base is None:
        return count_dict
    else:
        return count_dict[specific_base]

           

In [19]:
bases = count_bases(genome)
c_count = count_bases(genome, "C")

In [20]:
print(bases, c_count)

{'C': 15194, 'T': 7694, 'G': 15161, 'A': 7694} 15194
